# Linking Writing processes to writing quality

# Import the necessary packages needed for this analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from catboost import Pool, cv

In [ ]:
train_log=pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv')
train_log.head()

In [ ]:
train_scores=pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv')
train_scores.head()

In [ ]:
# lets read in the test data
test=pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv')
test.head()

In [ ]:
train_log.columns==test.columns

In [ ]:
train_log.shape, test.shape

 What I note is that the score is the same. That means that the score is for the final paper. The other variables keep changing.  Lets confirm this by looking at the second student 
 
 In light of this, we have to aggregate the data in terms of  each student. This will make it easy in terms  of the  final predictions. 

# Aggregating the data

## lets start with aggregating the numerical data

In [ ]:
# create a function that can be applied also to the test dataset

def aggregation(data):
    numeric = data.select_dtypes('int64')
    numeric['id'] = data['id']
    
    means = numeric.groupby('id').mean()
    stds = numeric.groupby('id').std()
    medians = numeric.groupby('id').median()
    mins = numeric.groupby('id').min()
    maxs = numeric.groupby('id').max()
    
    means.columns = [col + '_means' for col in means.columns]
    stds.columns = [col + '_stds' for col in stds.columns]
    medians.columns = [col + '_medians' for col in medians.columns]
    mins.columns = [col + '_mins' for col in mins.columns]
    maxs.columns = [col + '_maxs' for col in maxs.columns]
    result = pd.concat([means, stds, medians, mins, maxs], axis=1)
    
    return result

    

In [ ]:




def character_aggregation(data):
    character_df = data.select_dtypes('object')
    character_df['id'] = data['id']
    result=character_df.groupby('id').nunique()
    
    
    return result

In [ ]:
# now we creatfe a final function
def feature_extraction(data):
    numeric_df_clean= aggregation(data)
    character_df_clean=character_aggregation(data)
    result = numeric_df_clean.merge(character_df_clean, on='id', how='inner')
    return  result
train=feature_extraction(train_log)
    

In [ ]:
train.head()

In [ ]:
y=train_scores['score']
X=train

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2)

# Fitting a Stacking Regressor with catboost

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor

# Define the base models
base_models = [
    ('catboost', CatBoostRegressor(depth=6, iterations=500, learning_rate=0.01)),
    ('cat',  CatBoostRegressor()),
    ('RF3',RandomForestRegressor()),
    ('svr', XGBRegressor()),
    ('xgb', XGBRegressor(n_estimators=200)),
   ( 'rf', RandomForestRegressor()),
   ('xg1',XGBRegressor(n_estimators=300)),
   ('xg2', XGBRegressor(n_estimators=400)),
    ('xg3',XGBRegressor(n_estimators=500))
]
# Define the final estimator
final_estimator = SVR()  
# Create the stacking regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=final_estimator
)
# Fit the stacking regressor with your training data
stacking_regressor.fit(X_train, y_train)
# Make predictions and evaluate the model
y_pred = stacking_regressor.predict(X_test)


In [ ]:
y_preds=stacking_regressor.predict(X_test)
mean_absolute_error(y_test, y_preds)

In [ ]:
test_data=feature_extraction(test)
preds=stacking_regressor.predict(test_data)
preds

In [ ]:
submission=pd.DataFrame({'id':test_data.index, 'score':preds})
submission.to_csv('submission.csv', index=False)